##### - DNN기반 회귀 구현
 - 데이터셋 : IRIS.csv
 - 피쳐/속성 : 3개 Setal_Length, Sepal_Width, Petal_Width
 - 타겟/라벨 : 1개 Petal_Length
 - 학습-방법 : 지도학습 > 회귀
 - 알고 리즘 : 인공신경망(ANN) -> 심층신경망(MLP), DNN : 은닉층이 많은 구성
 - 프레임워크 : Pytorch

[1] 모듈 로딩과 데이터 준비 <hr>

In [83]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary
 
# Data 및 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [84]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수로 구현하기
print(f'torch v.{torch.__version__}')
print(f'Pandas v.{pd.__version__}')

torch v.2.4.1
Pandas v.2.0.3


In [85]:
print(dir())

['BATCH_SIZE', 'Binarizer', 'CNT', 'DATA_FILE', 'DEVICE', 'DataLoader', 'Dataset', 'EPOCH', 'F', 'FunctionTransformer', 'In', 'IrisDataset', 'IrisRegModel', 'KBinsDiscretizer', 'KernelCenterer', 'LOSS_HISTORY', 'LR', 'LabelBinarizer', 'LabelEncoder', 'MaxAbsScaler', 'MeanSquaredError', 'MinMaxScaler', 'MultiLabelBinarizer', 'Normalizer', 'OneHotEncoder', 'OrdinalEncoder', 'Out', 'PolynomialFeatures', 'PowerTransformer', 'QuantileTransformer', 'R2Score', 'RobustScaler', 'SCORE_HISTORY', 'SplineTransformer', 'StandardScaler', 'TargetEncoder', '_', '_14', '_22', '_25', '_30', '_37', '_40', '_45', '_53', '_56', '_6', '_61', '_63', '_69', '_72', '_77', '_9', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '__vsc_ipynb_file__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i29', '_i3', '_i30', '_i31'

In [86]:
def ver_check():
    import_list = list(dir())
    for i in import_list:
        try:
            
            print(f'{i} v.',i.__version__)
        except:
            print(i)

ver_check()

In [87]:
# import_list = list(dir())
# for i in import_list:
#     print(i.__version__)

In [88]:
### 데이터로딩
DATA_FILE = 'iris.csv'

### CSV => DataFrame
irisDF = pd.read_csv(DATA_FILE, usecols=[0,1,2,3])

### 확인
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


[2] 모델 클래스 설계 및 정의 <hr>
 - 클래스목적 : iris 데이터를 학습 및 추론 목적 
 - 클래스이름 : IrisRegModel
 - 부모클래스 : nn.Module
 - 매개__변수 : 층별 입출력 개수 고정하기 때문에 필요없음!
 - 속성__필드 : featureDF, targetDF, n_rows, n_features 
 - 기능  역할 : __init__()  : 모델 구조 설정, forward() : 순방향 학습, <= 오버라이딩(overriding)
 - 클래스구조 
    * 입력층 : 입력 3개 (피쳐)   출력 10개 (퍼셉트론 / 뉴런 10개 존재)
    * 은닉층 : 입력 10개         출력 30개 (퍼셉트론 / 뉴런 30개 존재)
    * 출력층 : 입력 30개         출력 1개 (너비값)


 - 활성화함수
   * 클래스 형태 ==> nn.MESLoss, nn.ReLU ==> __init__() 메서드
   * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드

In [89]:
class IrisRegModel(nn.Module):
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(3,10)
        self.hidden_layer = nn.Linear(10,30)
        self.out_layer = nn.Linear(30,1)


    def forward(self, input_data):
        y = self.in_layer(input_data)   # f11w11 + f12w12 + f13w13 ... + f103w103 + b
        # 여러 정보를 저장하는 배열
        # 활성화 함수에 들어가야지?
        y = F.relu(y)                       # relu => y 값의 범위 : 0 <= y  // 시그모이드는 0 ~ 1이긴 해.

        # 10개의 숫자를 갖고 은닉층 출동
        # - 은닉층 : 10개의 숫자 값 (>= 0)
        y = self.hidden_layer(y)                # f21w11 + f22+w12 + f23w13 ... f210w210 + b , f230w201 + f230w202 ... f230w230+b  
        y = F.relu(y)                       # 2번의 1번부터 시작, 2번의 30번까지.

        # - 출력층 : 30개의 숫자 값 (>=0)
        return self.out_layer(y)                   # f31w31 .... f330w330 + b   // 3번의 30번
        # 회귀이므로 바로 반환        

In [90]:
### 모델 인스턴스
model = IrisRegModel()



In [91]:
### 모델 사용 메모리 정보 확인
summary(model, input_size=(10,3))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [10, 1]                   --
├─Linear: 1-1                            [10, 10]                  40
├─Linear: 1-2                            [10, 30]                  330
├─Linear: 1-3                            [10, 1]                   31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

[1] 데이터셋 클래스 설계 및 정의 <hr>
 - 데이터셋 : iris.csv
 - 피쳐_개수 : 3개
 - 타겟_개수 : 1개
 - 클래스이름 : IrisDataset
 - 부모클래스 : utils.data.Dataset
 - 속성_필드 : featureDF, targetDF, n_rows, n_features
 - 필수 메서드
    * __init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * __len__(self) : 데이터 개수 반환
    * __getitem__(self, index): 특정 인덱스의 피쳐와 타겟 반환

In [92]:
class IrisDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]
        
    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화

        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values) # 그대로 가져오면 시리즈니까 values로 형 변환
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)
        # 피쳐와 타겟 변환

        return featureTS, targetTS

In [93]:
## 데이터셋 인스턴스 생성
# - DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]] # 2D (150,1)
targetDF = irisDF[irisDF.columns[-1:]]  # 2D (150,1)

# 커스텀데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF, targetDF)

[4] 학습 준비
 - 학습 횟수 : EPOCH            <- 처음 ~ 끝까지 공부하는 단위
 - 배치 크기 : BATCH_SIZE       <- 한번에 학습할 데이터셋 양
 - 위치 지정 : DEVICE           <- 텐서 저장 및 실행 위치 (GPU / CPU)
 - 학_습_률 : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001 ~ 0.1

In [94]:
### 학습 진행 관련 설정
EPOCH = 1
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

# print(f'BATCH_CNT : {BATCH_CNT}')

 - 인스턴스 / 객체 : 모델, 데이터셋, 최적화 (, 손실함수 , 성능지표)

In [95]:
# 모델 인스턴스
model = IrisRegModel()

# # 데이터셋 인스턴스
irisDS = IrisDataset(featureDF,targetDF)

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)


trainDS = IrisDataset(X_train,y_train)
valDS = IrisDataset(X_val, y_val)
testDS = IrisDataset(X_test,y_test)

trainDL = DataLoader(irisDS, batch_size=BATCH_SIZE)



# # 데이터로더 인스턴스
irisDL = DataLoader(irisDS, batch_size=BATCH_SIZE)

In [96]:
irisDL.dataset.n_features, irisDL.dataset.n_rows

(3, 150)

In [97]:
## [테스트] 데이터로더
for feature, target in irisDL:
    print(feature.shape, target.shape)
    break

torch.Size([10, 3]) torch.Size([10, 1])


In [98]:
# 최적화 인스턴스 => W,b 텐서 즉, model.parameters() 전달
optimizer = optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 회귀, MSE, MAE, RMSE ....
reqloss = nn.MSELoss()


[5] 학습 진행

In [106]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]], [[],[]]
CNT = irisDS.n_rows/BATCH_SIZE

for epoch in range(EPOCH):
    # 학습 모드로 모델 설정
    model.train()

    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    score_total, loss_total = 0,0
    for featureTS, targetTS in irisDL:
        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산
        loss = reqloss(pre_y, targetTS)
        loss_total += loss.item()
        # 성능평가 개선
        score = R2Score()(pre_y,targetTS)
        score_total += score.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()


    with torch.no_grad():
        # 검증 데이터셋 
        # 일부 뽑기 ....을마나 뽑을까 ... ㅎㅎ,ㅔ헤헤ㅔ
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)
        # 평가
        pre_val = model(val_featureTS)
        # 손실
        loss_val = reqloss(pre_val, val_targetTS)
        # 성능평가
        score_val = R2Score()(pre_val, val_targetTS)

    LOSS_HISTORY[0].append(loss_total/CNT)
    SCORE_HISTORY[0].append(score_total/CNT)
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

In [101]:
val_featureTS = torch.FloatTensor(valDS.featureDF.values)
val_targetTS = torch.FloatTensor(valDS.targetDF.values)

val_featureTS.shape, val_targetTS.shape

(torch.Size([28, 3]), torch.Size([28, 1]))

In [ ]:
print(f'CNT : {CNT}')